***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke pgem for 10 random seeds: range(10)

get pgem accuracy (+- std dev), wacc and tau

save in results/spammer_type/pgem.csv
***

## Device Setup

In [1]:
!nvidia-smi

Sun Dec 21 08:24:14 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   52C    P0              67W / 300W |    529MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


### Importing PGEM

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
from pgem import EMWrapper

## Passage dataset

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/passage/data/passage_cleaned.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,been wicked. They believed that the end of the...,lichen Sect Content Linking Artid A snake coil...,lichen Sect Content Linking Artid A snake coil...,5
20,"school, you noticed that all the clocks read a...",man in the seat with me and the two women acro...,man in the seat with me and the two women acro...,5
19,foreign animals or plants may be taken into th...,adhere to the wall. Using sizing that has been...,foreign animals or plants may be taken into th...,5
18,many pennies did each child get Which computat...,the picture on its cover. Write two or three s...,many pennies did each child get Which computat...,5
17,direction of the Sun. The length in meters and...,"the ground going into the cave. Oh, my he said...","the ground going into the cave. Oh, my he said...",5


In [7]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [8]:
import pickle

with open("../../real_data/passage/data/PassageDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,label,score
0,"a star. Our planet, Earth, orbits, or circles,...",1
1,"Adam, We did not have plastic toys. I played w...",1
2,Who said the little owl. Who wants to hunt wit...,1
3,dead leaf. This is a mole. Moles burrow underg...,1
4,ereaddatagradepsenvironcomp.html Environment r...,1
...,...,...
467,work over the summer on any changes they wish ...,12
468,between January and December plunged the Unite...,12
469,into a newly opened bank account. I was amazed...,12
470,"occurring phenomenon, manmade by products are ...",12


In [9]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

472


In [10]:
gt_df = df_passage

### Addition of Random Guessors

In [11]:
spammer_type = "random"

In [12]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

Unique performers: 686
686
cuda
cuda


  1%|█▏                                                                                                                     | 1/100 [00:00<01:03,  1.57it/s]

Iter 000: Log-likelihood = -0.472055


 24%|████████████████████████████▎                                                                                         | 24/100 [00:08<00:26,  2.92it/s]


Converged at iter 24, Log-likelihood change = 9.536743e-07
cuda
cuda


...

 27%|███████████████████████████████▊                                                                                      | 27/100 [00:11<00:30,  2.36it/s]


Converged at iter 27, Log-likelihood change = 8.642673e-07
cuda
cuda


  1%|█▏                                                                                                                     | 1/100 [00:00<00:41,  2.37it/s]

Iter 000: Log-likelihood = -0.472096


 15%|█████████████████▋                                                                                                    | 15/100 [00:05<00:30,  2.82it/s]

### Addition of Anti-Personas

In [ ]:
spammer_type = "anti"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

### Addition of Left Position Biased Spammers

In [ ]:
spammer_type = "left"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="left", seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

### Addition of Right Position Biased Spammers

In [ ]:
spammer_type = "right"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="right", seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

### Addition of Equal Proportion of Spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")